### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [ ]:
%run "../includes/configurations"

In [ ]:
%run "../includes/common_functions"

In [ ]:
appName = "Pit Stops Ingestion"
spark = create_spark_context(appName, warehouse_location)

In [ ]:
pit_stops_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/pit_stops")

### Step 2 - Rename and add new columns

In [ ]:
pit_stops_column_mapping = {
    "raceId": "race_id",
    "driverId": "driver_id"
}

pit_stops_df = rename_columns(pit_stops_df, pit_stops_column_mapping)

In [ ]:
pit_stops_df = add_ingestion_date(pit_stops_df)

### Step 3 - Write data to a Datalake as a Parquet

In [ ]:
pit_stops_df.write.mode("overwrite").parquet(f"{processed_folder_path}/pit_stops")